# <B> Clean up </B>

## AutoReload

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## clean

In [9]:
import os
import time
import boto3
from utils.ssm import parameter_store

In [6]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

* S3 Bucket

In [10]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))

In [11]:
strQuery = "".join(["aws s3 rm --recursive ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()
strQuery = "".join(["aws s3 rb ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()

* endpoint

In [12]:
class clean_up():
    
    def __init__(self, ):    
        pass
    
    def delete_endpoint(self, client, endpoint_name ,is_del_model=True):
        
        response = client.describe_endpoint(EndpointName=endpoint_name)
        EndpointConfigName = response['EndpointConfigName']

        response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
        model_name = response['ProductionVariants'][0]['ModelName']    

        if is_del_model: # 모델도 삭제 여부 임.
            client.delete_model(ModelName=model_name)    

        client.delete_endpoint(EndpointName=endpoint_name)
        client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    

        print(f'--- Deleted model: {model_name}')
        print(f'--- Deleted endpoint: {endpoint_name}')
        print(f'--- Deleted endpoint_config: {EndpointConfigName}')  

In [17]:
clean = clean_up()
sm_client = boto3.client('sagemaker')

## 2.training 
strEndPointName = pm.get_params(key="-".join([strPrefix, "ENDPOINT-NAME-DEPLOY"]))
clean.delete_endpoint(sm_client, strEndPointName ,is_del_model=True)

## 4.pipeline
strEndPointName = pm.get_params(key="-".join([strPrefix, "ENDPOINT-NAME"]))
clean.delete_endpoint(sm_client, strEndPointName ,is_del_model=True)

--- Deleted model: sagemaker-xgboost-2023-04-16-06-51-37-705
--- Deleted endpoint: endpoint-local-DJ-SM-IMD1681627890
--- Deleted endpoint_config: endpoint-local-DJ-SM-IMD1681627890
--- Deleted model: sagemaker-xgboost-2023-04-16-07-29-30-429
--- Deleted endpoint: endpoint--DJ-SM-IMD-PIPELINE1681629485
--- Deleted endpoint_config: endpoint--DJ-SM-IMD-PIPELINE1681629485


* pipeline

In [20]:
strPipelineName = pm.get_params(key="-".join([strPrefix, "PIPELINE-NAME"]))
strPipelineName

IndexError: list index out of range

In [22]:
!aws sagemaker delete-pipeline --pipeline-name $strPipelineName 

{
    "PipelineArn": "arn:aws:sagemaker:us-east-1:419974056037:pipeline/dj-sm-imd-pipeline"
}


* model registry

In [23]:
sagemaker_client = boto3.client('sagemaker')

In [24]:
def empty_and_delete_model_package(sagemaker_client, mpg_name):
    mpg = sagemaker_client.list_model_packages(
        ModelPackageGroupName=mpg_name,
    )
    
    # Delete model packages if Group not empty
    model_packages = mpg.get('ModelPackageSummaryList')
    if model_packages:
        for mp in model_packages:
            sagemaker_client.delete_model_package(
                ModelPackageName=mp['ModelPackageArn']
            )
            time.sleep(1)

    # Delete model package group
    sagemaker_client.delete_model_package_group(
        ModelPackageGroupName=mpg_name
    )

In [25]:
DeleteModelGroupName = pm.get_params(key=strPrefix + "MODEL-GROUP-NAME")
empty_and_delete_model_package(sagemaker_client, DeleteModelGroupName)

IndexError: list index out of range

* parameter store

In [26]:
listParams = pm.get_all_params()
listParams = [strParam for strParam in listParams if strPrefix in strParam] + ["PREFIX"]

total_size = len(listParams)
for idx in range(0, total_size, 10):
    start = idx
    if start + 10 < total_size: end = start + 10
    else: end = total_size
    listSubParams = listParams[start:end]
    pm.delete_param(listSubParams)

  parameters: ['DJ-SM-IMD-ACCOUNT-ID', 'DJ-SM-IMD-BUCKET', 'DJ-SM-IMD-CODE-BUILD-ROLE-ARN', 'DJ-SM-IMD-CODE-PIPELINE-ROLE-ARN', 'DJ-SM-IMD-END-POINT', 'DJ-SM-IMD-ENDPOINT-NAME', 'DJ-SM-IMD-EVAL-OUTPUT-PATH', 'DJ-SM-IMD-EXPERI-NAME', 'DJ-SM-IMD-LAMBDA-ROLE-ARN', 'DJ-SM-IMD-MODEL-PATH'] is deleted successfully
  parameters: ['DJ-SM-IMD-PIPELINE-ARN', 'DJ-SM-IMD-REGION', 'DJ-SM-IMD-SAGEMAKER-ROLE-ARN', 'PREFIX'] is deleted successfully


* code commit, buldand pipeline

In [15]:
from utils.code_pipeline import code_pipeline_handler
cph = code_pipeline_handler()

In [16]:
cph.delete_repository(strRepoName="SM-MLOPS-PIPELINE-DEMO")
cph.delete_build_project(strCodeBuildPJTName="demo-mlops-build")
cph.delete_pipeline(strPipeLineName="demo-mlops-code-pipeline")

== DELETE REPO ==
  Repository name [SM-MLOPS-PIPELINE-DEMO] was successfully deleted!!
''
''
